# Integração de Monte Carlo

## $ \S 1 $ Introdução

Nas regras do trapézio e de Simpson, escolhemos $ N + 1 $ pontos uniformemente distribuídos no intervalo de integração $ [a, b] $ como membros da "amostra" tomada para os valores da função $ f $ a ser integrada. Uma outra estratégia é escolher os pontos onde $ f $ será avaliada *aleatoriamente* dentro de $ [a, b] $. 

Os métodos do primeiro tipo são chamados de **determinísticos**. Uma vez fixados os parâmetros (a função contínua $ f \colon [a, b] \to \mathbb R $ e o número $ N + 1 $ de pontos), cada aplicação do método fornece o mesmo resultado. Além disto, temos uma estimativa precisa do erro cometido em função de $ N $.

Em contraste, o método da *integração de Monte Carlo* que estudaremos neste caderno é **probabilístico**. Por envolver escolhas aleatórias, cada aplicação fornece uma aproximação diferente, mesmo mantendo-se $ N $ fixo. Ademais, não podemos garantir que o erro envolvido na aproximação seja pequeno para $ N $ grande, temos somente uma estimativa da *probabilidade* de que o erro não irá exceder um certo valor dado.

Na verdade a integração de Monte Carlo é um caso especial de um método muito mais geral e extremamente poderoso, o *método de Monte Carlo*.

 ## $ \S 2 $ Descrição do método de Monte Carlo
 
O **método de Monte Carlo** consiste da execução de um alto número $ N $ de *experimentos* ou *ensaios* num determinado contexto e da análise estatística dos resultados obtidos para se chegar a conclusões. 

Como exemplo, considere o disco $ D $ de equação
$$
x^2 + y^2 \le 1
$$
no plano $ \mathbb R^2 $, cuja área é $ \pi $ e que está contido no quadrado $ Q $ de área $ 4 $ descrito pelas desigualdades
$$
\lvert{x}\rvert \le 1, \quad \lvert{y}\rvert \le 1.
$$
A probabilidade de que um ponto escolhido aleatoriamente dentro deste quadrado esteja dentro do disco é o quociente entre as áreas, 
$$
\frac{\text{área}(D)}{\text{área}(Q)} = \frac{\pi}{4}.
$$
Suponha que escolhamos aleatoriamente $ 1, 2, 3, \dots $ até um total de $ N $ pontos dentro de $ Q $, e seja $ n $ o número destes pontos pertencentes ao disco $ D $. Um teorema da Teoria do Probabilidade chamado de **Lei dos Grandes Números** diz que o quociente entre o número de "sucessos" $ n $ e o número total $ N $ de pontos na amostra tende à probabilidade do sucesso:
$$
\lim_{N \to \infty} \frac{n}{N} = \frac{\pi}{4}.
$$
Mais ainda, dado $ \varepsilon > 0 $ positivo qualquer,
$$
\lim_{N \to \infty} \text{pr}\bigg(\Big\lvert \frac{n}{N} - \frac{\pi}{4}\Big\rvert < \varepsilon \bigg) = 1.
$$
Em palavras, a probabilidade de que a fração entre o número de pontos dentro do disco ($ n $) e o número total de pontos ($ N $) difira de $ \frac{\pi}{4} $ por mais que $ \varepsilon $ tende a zero conforme o número de experimentos aumenta!

In [34]:
def monte_carlo(ensaio, N):
    n = 0
    for i in range(N):
        if ensaio():
            n += 1
            
    return n / N

In [35]:
from numpy.random import rand


def pertence_ao_disco():
    # 'esticar' transforma o intervalo [0, 1] no intervalo [-1, 1]:
    esticar = lambda x: 2 * x - 1
    
    # Escolha números aleatórios x, y entre -1 e 1.
    # Recorde que rand() escolhe um float aleatório em [0, 1].
    x = esticar(rand())
    y = esticar(rand())
    
    return (x**2 + y**2 <= 1)


In [39]:
from numpy import pi


N = 10**4
for k in range(10):
    print(4 * monte_carlo(pertence_ao_disco, N))

print(pi)

3.1412
3.1568
3.13
3.1576
3.1356
3.1512
3.1432
3.1496
3.1528
3.1348
3.141592653589793


**Problema 1:** Um teorema de E. Cèsaro (1850—1906) diz que a probabilidade de que dois naturais escolhidos ao acaso sejam relativamente primos é de $ \frac{6}{\pi^2} $. (Dois inteiros $ m $ e $ n $ são *relativamente primos* caso $ \text{mdc}(m, n) = 1 $.)

(a) Construa um experimento que escolhe dois números aleatórios $ m $ e $ n $ entre $ 1 $ e $ 10^6 $ e retorna `True` caso eles sejam relativamente primos e `False` caso contrário. *Dica:* A função `randint(a, b)` do módulo `numpy.random` retorna um inteiro aleatório entre $ a $ e $ b - 1 $ (inclusive).

(b) Use o método de Monte Carlo e o teorema citado para obter uma aproximação para $ \pi $.

In [52]:
from numpy import sqrt, pi
from numpy.random import randint


def mdc(a, b):
    """Retorna o mdc de dois inteiros a, b. """
    assert isinstance(a, int) and isinstance(b, int)
    if b == 0:
        return a
    else:
        return mdc(b, a % b)

    
def ensaio_rel_primo():
    m = randint(1, 10**5)
    n = randint(1, 10**5)
    return (mdc(m, n) == 1)


N = 10**4
resultado = monte_carlo(ensaio_rel_primo, N)
print(sqrt(6 / resultado))
print(pi)

3.1510067105223283
3.141592653589793


## $ \S 3 $ Aplicação a integrais de funções de uma variável

Recorde do curso de Cálculo 1 que a integral
$$
\int_a^b f(x)\,dx
$$
é aproximada por *somas de Riemann*
$$
\sum_{i=1}^N f(\xi_i) \Delta_i
$$
onde
$$
[x_0, x_1],\ [x_1, x_2],\ \dots,\ [x_{N-1}, x_{N}]
$$
formam uma partição de $ [a, b] $,
$$
\Delta_i = x_i - x_{i-1}
$$
e os pontos
$$
\xi_i \in [x_{i-1}, x_i] \qquad (i = 1, \dots, N)
$$
são escolhidos arbitrariamente. De fato, a integral é *definida* como o limite de somas deste tipo conforme o maior dos comprimentos $ \Delta_i $ vai a zero.

Considere o caso especial em que $ \Delta_i = \frac{b-a}{N} $ para cada $ i $, ou seja, os pontos são
$$
x_i = a +  i\frac{b-a}{N}
$$
e estão igualmente espaçados. Então a soma de Riemann se reduz a 
$$
\frac{b-a}{N}\sum_{i=1}^N f(\xi_i) = (b - a)\bigg(\frac{1}{N}\sum_{i=1}f(\xi_i)\bigg).
$$    
O termo entre parênteses aqui pode ser visto como a média de uma amostra de $ N $ valores de $ f $ no intervalo, sujeitos à condição de que o $ i $-ésimo ponto pertence ao $ i $-ésimo intervalo da partição. A idéia da **integração de Monte Carlo** é aproximar $ \int_a^b f(x)\,dx $ pelo valor à direita, sem impor restrições sobre os $ \xi_i $ (exceto que eles devem pertencer à $ [a, b] $).

In [54]:
def integracao_monte_carlo(f, a, b, N):
    from numpy.random import rand
    
    # Transforma o intervalo [0, 1] no intervalo [a, b]:
    esticar = lambda x: a + (b - a) * x
    
    soma = 0
    for i in range(N):
        x = esticar(rand())
        soma += f(x)
    
    return soma / N

In [56]:
from numpy import pi


f = lambda x: 1 / (1 + x**2)
a = 0
b = 1
N = 10000

print(pi)
print(4 * integracao_monte_carlo(f, a, b, N))

3.141592653589793
3.141867745364274
